In [2]:
import matplotlib.pyplot as plt


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd() / "_scripts"))

In [236]:
import networkx as nx
from eppy.bunch_subclass import EpBunch
from geomeppy import IDF
from geomeppy import extractor
import seaborn as sns
import polars as pl
import numpy as np
from dataclasses import dataclass

from case_edits.ezcase import *
from helpers.helpers import *
from helpers.variables import *
from helpers.variable_interfaces import *
from setup.setup import *
from setup.plots import *
from setup.data_wrangle import *
from analysis.data_on_network import *

import matplotlib.cm as cm
import matplotlib as mpl


from matplotlib.colors import Colormap, Normalize, LinearSegmentedColormap

from helpers.ep_helpers import *
from subsurfaces.creator import SubsurfaceCreator
from airflow_network.airboundary import *
from subsurfaces.logic import find_surface_connecting_two_zones
from analysis.dataframes import *

from eppy.constructions import thermal_properties
from constructions.constructions import *
from constructions.unified_constructions import create_unified_constructions_idf
import re


In [241]:
get_cons_set_by_type("Light")

[
 Construction,
     Light Exterior Wall,      !- Name
     F08 Metal surface,        !- Outside Layer
     I02 50mm insulation board,    !- Layer 2
     F04 Wall air space resistance,    !- Layer 3
     G01a 19mm gypsum board;    !- Layer 4,
 
 Construction,
     Light Roof/Ceiling,       !- Name
     M11 100mm lightweight concrete,    !- Outside Layer
     F05 Ceiling air space resistance,    !- Layer 2
     F16 Acoustic tile;        !- Layer 3,
 
 Construction,
     Light Partitions,         !- Name
     G01a 19mm gypsum board,    !- Outside Layer
     F04 Wall air space resistance,    !- Layer 2
     G01a 19mm gypsum board;    !- Layer 3,
 
 Construction,
     Light Floor,              !- Name
     F16 Acoustic tile,        !- Outside Layer
     F05 Ceiling air space resistance,    !- Layer 2
     M11 100mm lightweight concrete;    !- Layer 3,
 
 Construction,
     Light Furnishings,        !- Name
     G05 25mm wood;            !- Outside Layer,
 
 Construction,
     Light Window

In [156]:
# cases = retrieve_cases(AIRBOUNDARY_FOLDER)
# case = cases[0]

case = retrieve_test_case("test25_airwall")
case

CaseData(case_name='test', idf=<geomeppy.idf.IDF object at 0x1105aa890>, sql=Energy SQLiteResult: /Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/cases/tests/test25_airwall/results/eplusout.sql, path_to_input=PosixPath('/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/svg2plan/outputs2/case_bol_5'), path_to_output=PosixPath('/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/cases/tests/test25_airwall'))

In [237]:
idf = create_unified_constructions_idf()
idf

In [240]:
pattern = re.compile("Sgl Clr \dmm")
wcons = [i for i in idf.idfobjects["CONSTRUCTION"] if pattern.findall(i.Name)]
wcons


[
 Construction,
     Light Window Sgl Clr 3mm,    !- Name
     CLEAR 3MM;                !- Outside Layer,
 
 Construction,
     Medium Window Sgl Clr 6mm,    !- Name
     CLEAR 6MM;                !- Outside Layer,
 
 Construction,
     Heavy Window Sgl Clr 3mm,    !- Name
     CLEAR 3MM;                !- Outside Layer]

In [235]:
wcons = [i for i in idf.idfobjects["CONSTRUCTION"] if pattern.match(i.Name)]


new_names = [f"{t} Window {i.Name}" for t,i in zip(["Light", "Medium", "Heavy"] ,wcons)]
for i, n in zip(wcons, new_names):
    i.Name = n

wcons

[
 Construction,
     Light Window Sgl Clr 3mm,    !- Name
     CLEAR 3MM;                !- Outside Layer,
 
 Construction,
     Medium Window Sgl Clr 6mm,    !- Name
     CLEAR 6MM;                !- Outside Layer,
 
 Construction,
     Heavy Window Sgl Clr 3mm,    !- Name
     CLEAR 3MM;                !- Outside Layer]

In [243]:
idf = assign_cons_set(case.idf, "Light")

In [244]:
idf.getsubsurfaces()

[
 DOOR:INTERZONE,
     Block 06 Storey 0 Wall 0002_1 Door,    !- Name
     Light Furnishings,        !- Construction Name
     Block 06 Storey 0 Wall 0002_1,    !- Building Surface Name
     Block 06 Storey 0 Wall 0002_1 Door Partner,    !- Outside Boundary Condition Object
     1.0,                      !- Multiplier
     3.650000e-02,             !- Starting X Coordinate
     3.050000e-02,             !- Starting Z Coordinate
     0.6570000004023314,       !- Length
     2.744999999832362;        !- Height,
 
 DOOR:INTERZONE,
     Block 06 Storey 0 Wall 0002_1 Door Partner,    !- Name
     Light Furnishings,        !- Construction Name
     Block 00 Storey 0 Wall 0004_2,    !- Building Surface Name
     Block 06 Storey 0 Wall 0002_1 Door,    !- Outside Boundary Condition Object
     1.0,                      !- Multiplier
     3.650000e-02,             !- Starting X Coordinate
     3.050000e-02,             !- Starting Z Coordinate
     0.6570000004023314,       !- Length
     2.744

In [165]:
path_to_constructions = Path.cwd() / "cases" / "constructions"
cidf = IDF(path_to_constructions / "ASHRAE_2005_HOF_Materials.idf")
new_idf = IDF(path_to_constructions / "CombinedConstructs.idf")
extractor.copy_constructions(cidf, new_idf)

In [205]:

# new_idf.touch()
new_idf

In [166]:
win_cons_idf = IDF(path_to_constructions / "WindowConstructs.idf")
win_glass_mat_idf = IDF(path_to_constructions / "WindowGlassMaterials.idf")
augmented_win_cons_idf = extractor.copy_constructions(win_glass_mat_idf, win_cons_idf)

In [167]:

win_glass_mat_idf

In [ ]:
win_glass_mat_idf.idfobjects["WindowMaterial:Glazing".upper()]

In [185]:
augmented_win_cons_idf = extractor.copy_constructions(win_glass_mat_idf, win_cons_idf)


In [200]:
import re
from constructions.materials import get_construction_set_materials, get_construction_materials

In [196]:
pattern = re.compile("Sgl Clr \dmm")

In [198]:
wcons = [i for i in augmented_win_cons_idf.idfobjects["CONSTRUCTION"] if pattern.match(i.Name)]
wcons

[
 Construction,
     Sgl Clr 3mm,              !- Name
     CLEAR 3MM;                !- Outside Layer,
 
 Construction,
     Sgl Clr 6mm,              !- Name
     CLEAR 6MM;                !- Outside Layer]

In [212]:
new_names = [f"{t} Window {i.Name}" for t,i in zip(["Light", "Heavy"] ,wcons)]
for i, n in zip(wcons, new_names):
    i.Name = n

In [213]:
wcons

[
 Construction,
     Light Window Sgl Clr 3mm,    !- Name
     CLEAR 3MM;                !- Outside Layer,
 
 Construction,
     Heavy Window Sgl Clr 6mm,    !- Name
     CLEAR 6MM;                !- Outside Layer]

In [214]:
med_window = new_idf.copyidfobject(wcons[0])
med_window.Name = 


Construction,
    Light Window Sgl Clr 3mm,    !- Name
    CLEAR 3MM;                !- Outside Layer

In [204]:
# find related objects.. 
wmats = get_construction_set_materials(wcons)


In [207]:
res = wcons + wmats
for i in res:
    new_idf.copyidfobject(i)
    

In [208]:
new_idf.save()

In [ ]:
# extract main constructions. 